In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


<h1 style='background:#26A2AB; border:0; color:black'><center>Our job will be structured in this way:</center></h1> 



1) First of all let's read the datasets of training and test and visualize it to have in a glance the first idea of what our datasets consists in.

2) Make some data cleaning searching if we have outliers and missing values and than make some feature engineering if we found something strange.

3) Using some visualizations to search some further insight that we couldn't see before just reading the data.

4) Preparing the data for the model with scaling or normalization and splitting it in train and validation set

5) Apply some regression beginnning from simple linear regression understanding the general interpretability of the model and using after that some regularizations

6) Try other kind of models different from linear Regression and choose which one could be the best to get a better interpretation of the data for this case


READ THE DATA
---

In [ ]:
train=pd.read_csv("../input/tabular-playground-series-jan-2021/train.csv")

In [ ]:
train.head()

In [ ]:
test=pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv")

In [ ]:
test.head()

In [ ]:
submission=pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv")

In [ ]:
submission

I want to see the general information of both train and test data
---

In [ ]:
train.shape,test.shape

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:

#We can see in the previous table that the distributions of the variables for training and testing set are quite similar
a=train.describe()
b=test.describe()
a-b


It was just to have a glance and verify that the distributions of our features are quite similar for the 2 datasets

In [ ]:
train.shape,test.shape

Our datasets are composed of 30k records for the train that we will use to train our final model and 20k records withouth the target variable for the test dataset. Our task for this competition is just to predict the value for the target variable.
That's why the number of column is different. The only differene is in the variable to predict.

EXPLORATORY DATA ANALYSIS
===

we have different ooerations to do before to start:

- check the missing values
- find the outliers
- study the correlations and drop unuseful variables
- some visualizations
- make some transformations if needed like changing.



Check of the missing Values
---
We have no missing values for both the datasets. 
The cell below show the code to visualize the number of null values in the different columns of the 2 datasets.

We have no missing values for both the datasets

In [ ]:
train.isnull().sum(),test.isnull().sum()

A first blick to the target variable
===

Let's see the distribution of the target variable. As we can imagine the nature is numeric continuous and the distribution seems to be bell shaped. We can see the distribution in form of histograms.
While this variable is not continuous the histogram could be misleading aniway.

In [ ]:
train.target.hist()

If we increase the number of bins we can realize that we have a bimodal distribution and it's not really well shaped. 
We could use some specific test to evaluate the normality of the variable and i'll leave some code later about  how to verify it.

In [ ]:
train.target.hist(bins =20)

With the nest command we can have a better idea using the distplot of which is the real shape of the distribution.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.distplot(train.target)

Check the normality of the curve
===
We want to see if the target variable is normally distributed and we can find inspiratio from this link triyng some useful tests of normality:
https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/

In [ ]:
from scipy import stats
a=train.target
a=np.array(a)
k2,p=stats.normaltest(a)

In [ ]:
alpha = 1e-3
print("p = {:g}".format(p))
if p < alpha:  # null hypothesis: x comes from a normal distribution
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")

you can see more about this test (scipy.normaltest) on this link:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html

OUTLIERS
===

To detect the outliers the best strategy is using the boxplots

In [ ]:
plt.boxplot(train.target)
plt.show()

In [ ]:
train.target.describe()

We didn't see before some ourliers in the distribution. There is a suspicious value of 0 for the target value that is probably an outlier. Let's check it out below the sorted extreme values.
We could in case cut this observation from the train data if we think it can affect our final prediction, just we should try to understand if is an aberrant value or a real outlie. The same with the extreme  values of the distribution. 
The dataset is aniway enough large so it shouldn't be a problem to cut the zero value and some extreme observations.

In [ ]:
train.target.sort_values()

Dropping the outlier for the target variable
---

In [ ]:
train.drop(index=170514,inplace=True)
train.target.min()

Outliers for the predictors
---
Now we can search the outliers in the the other variables.
With the block of code below we can simoultaneuosly control it for both the datasets and all the feature except the target variable.

In [ ]:
for col in train.columns[:-1]:
    plt.boxplot([train[col],test[col]], labels=['train', 'test'])
    plt.title(col)
    plt.legend()
    plt.show()

For this outliers treatment i took inspiration by the work of Gaurav Rajesh Sahani:
https://www.kaggle.com/code/gauravsahani/detailed-eda-with-lightgbm-score-0-698

In [ ]:
Q1_train = train.quantile(0.25)
Q3_train = train.quantile(0.75)
IQR_train = train - Q1_train
((train < train - 1.5*IQR_train) | (train > Q3_train + 1.5*IQR_train)).agg([sum, 'mean'])

****We can find outliers for the variables cont7 and cont9 as well as we saw for the target variable. One possible strategy could be to replace it with the median or the 1st quartile of the distribution or triyng to estimate it with a model of regression. For the moment we keep it invariate. Indeed the outliers when present are both in the train and in the test data. If i had outliers just in training data i'd drop that rows but it's not the case.

In [ ]:
Q1_test = test.quantile(0.25)
Q3_test = test.quantile(0.75)
IQR_test = Q3_test - Q1_test
((test < Q1_test - 1.5 * IQR_test) | (test > Q3_test + 1.5 * IQR_test)).agg([sum, 'mean'])

We will create a function to remove the outliers:

In [ ]:
def replace_outliers(data):
    for col in data.columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        median_ = data[col].median()
        data.loc[((data[col] < Q1 - 1.5*IQR) | (data[col] > Q3 + 1.5*IQR)), col] = median_
    return data

In [ ]:
train=replace_outliers(train)

In [ ]:
test=replace_outliers(test)

Multivariate analysis of the dataset
===
Let's start from t the correlations between the features.
We will check it by correlation matrix and heatmap for the train dataset.
We could visualize it with a simply line of comman (train.corr()) but if we want a more intuitive version of the correlation matrix we can add some detail about the style.

In [ ]:
corr=train.corr()
corr.style.background_gradient(cmap="coolwarm").set_precision(2)

We can cat the correlation matrix inside a heatmap with the use of masks

The first thing to look is the last row of the table. WE can observe any particularly strong correlation between thetarget variable and the other while we can observe many high correlations between the other continuous variable that can lead to a problem of multicollinearity in the data.
In the next cell we execute the command to highlights the correlations with the use of the heatmap

In [ ]:
#we create a mask to visualize values just once
plt.figure(figsize=(14,8))
mask=np.zeros_like(corr)
mask[np.triu_indices_from(mask,1)]=True

plt.subplots(figsize=(15,12))
sns.heatmap(train.corr(),mask=mask,vmax=0.9,cmap="viridis",annot=True, square =True)
plt.show()

This table show what we note before about the lack of strong correlation between target variable and the predictors

In [ ]:
train.corr()[15:16].T

In [ ]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score

X=train.drop(["id"],axis=1)
y=train.target

I can drop the variable id that is not particularly significant in this case

In [ ]:
train=train.drop("id",axis =1)

Some visualizations about the single variables
===

In [ ]:
%matplotlib inline
train.hist(bins=10,figsize=(10,10))
plt.show()

In [ ]:
train.skew()

Many variables has problem of skewness. We could make some transformation as well to solve this problem if we find it necessary. In the previous cell of describe method we didn't find particularly problem about the different magnitude of the features so we should scale the data but in this case the situations shouldn't change too much. 

Pairplot
---

In [ ]:
#sns.pairplot(train)

Splitting the dataset for the validation
===

i want to create a categorical variable for the target with the aim to create a split of the dataset who proportionally represent the dataset

In [ ]:
X.head()

In [ ]:
X=X.drop("target",axis=1)
X

In [ ]:
%matplotlib inline
#grr=pd.plotting.scatter_matrix(X[0:80],c=y[0:80],figsize=(15,10))
#plt.show()

In [ ]:
X["cont5_13"]=X["cont5"]*X["cont13"]
X["cont7_10"]=X["cont7"]*X["cont10"]
X["cont1_5"]=X["cont1"]*X["cont5"]

test["cont5_13"]=test["cont5"]*test["cont13"]
test["cont7_10"]=test["cont7"]*test["cont10"]
test["cont1_5"]=test["cont1"]*test["cont5"]

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.3,random_state=42)

The mean for the 2 splitted vector of y are similar

In [ ]:
y_train.mean(),y_val.mean()

In [ ]:
y_train.describe()

In [ ]:
y_val.describe()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet, Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge

Creating a function to evaluate the cross-validation score

In [ ]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
lr_model=LinearRegression()
lr_model.fit(X_train,y_train)

With the command below we create a function that can help us to have an idea of the interpretation of the Linear Regression model and we will understand its utility soon.

In [ ]:
def linearRegressionSummary(model, column_names):
    '''Show a summary of the trained linear regression model'''

    # Plot the coeffients as bars
    fig = plt.figure(figsize=(8,len(column_names)/3))
    fig.suptitle('Linear Regression Coefficients', fontsize=16)
    rects = plt.barh(column_names, model.coef_,color="lightblue")

    # Annotate the bars with the coefficient values
    for rect in rects:
        width = round(rect.get_width(),4)
        plt.gca().annotate('  {}  '.format(width),
                    xy=(0, rect.get_y()),
                    xytext=(0,2),  
                    textcoords="offset points",  
                    ha='left' if width<0 else 'right', va='bottom')        
    plt.show()

In [ ]:
linearRegressionSummary(lr_model,X.columns)

We just used the previous function written above to have a visualization of the impact of the predictors in the Regression model. The ones with the bar on the left affected our prediction in negative way decreasing the value of the target ( in particular cont10 while the ones on the right influence the result in positive way increasing the value of the target (expecially cont6).
The more the bar is far from the centre the more the predictor has an influence in our model.

In [ ]:
print(rmsle_cv(lr_model))
score=rmsle_cv(lr_model)
print("\nmodel score: {:.4f} {:.4f}\n".format(score.mean(),score.std()))

In [ ]:
pred=lr_model.predict(X_val)

In [ ]:
res_df = pd.DataFrame({
    'Actual': y_val, 
    'Predicted':pred
})

res_df


We can have another view of our predictions with this cell of command

In [ ]:
df1 = res_df.head(25)
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:

test=test.drop("id",axis=1)
test.head()

In [ ]:
lr_model=LinearRegression()
lr_model.fit(X,y)
predictions=lr_model.predict(test)


In [ ]:
predictions

In [ ]:
print(lr_model.intercept_)
lr_model.coef_

In [ ]:
output=pd.DataFrame({"id":id,"target":predictions})
output
output.to_csv('sample_submission.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
output

1st little improving: Introducing scaling
===

In [ ]:
#Let's try to use scaling on the same model
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler


In [ ]:
rs = RobustScaler()
rs.fit(X_train)
X_train_rs = pd.DataFrame(rs.transform(X_train), columns=X_train.columns, index=X_train.index)
X_val_rs = pd.DataFrame(rs.transform(X_val), columns=X_val.columns, index=X_val.index)
X_rs=pd.DataFrame(rs.transform(X), columns=X_val.columns, index=X.index)

X_train.shape, X_train_rs.shape, X_val.shape, X_val_rs.shape,X_rs.shape

In [ ]:
X_train_rs.head()

In [ ]:
X_train.head()

In [ ]:
X_val_rs.head()

In [ ]:
X_val.head()

Further ideas to improve the model
===

There are some library that you could use to improve your model making some hyperparameter tuning like *optuna* for a specific model or trying to evaluate more model simoultaneously on the same dataset using the library *lazypredict*. 
feel free to try this 2 options for this dataset and evaluate the results.
for the moment i will use a pipeline for some of the best models available for this kind of problem like lightgbm,xgboost and random forest in general

In [ ]:
from sklearn.pipeline import Pipeline
def get_model():
    pipe = Pipeline( steps=[
        ("regressor", None)
    ])
    return pipe

In [ ]:
"""from sklearn.model_selection import  GridSearchCV
param_grid = {
    'regressor': [
        LinearRegression(),
        Ridge(alpha=0.5, random_state=42),
        Lasso(alpha=0.5, random_state=42),
        RandomForestRegressor(max_depth=5, random_state=42)
    ]
}

search = GridSearchCV(
    get_model(), 
    param_grid, 
    cv=KFold(n_splits=5, shuffle=True, random_state=123), 
    scoring="neg_mean_squared_error",
    return_train_score=True, 
    verbose=3
)
"""

In [ ]:
"""%%time
search.fit(X_rs, y)
"""

In [ ]:
#search.best_estimator_

The result was:
Pipeline(steps=[('regressor',
                 RandomForestRegressor(max_depth=5, random_state=42))])

In [ ]:
#search.best_score_

While the best result is obtained with the RandomForestRegressor i can decide to oveercome the problem of transformers because is not affected by problem of scaling so i'll apply on the X dataset. It doesn't mean that this is the best model in absolute... we could try something more extreeme like lightgbm as well , but it's the best between the four models tested for this problem.
We can still improve it using the GridSearchCV for the same model changing some parameters like the maximum depth or the number of leaf in the trees. 

RandomForestModel
===

In [ ]:
"""lasso=make_pipeline(MinMaxScaler(),Lasso(alpha=0.0001,random_state=42))
print(rmsle_cv(lasso))
score=rmsle_cv(lasso)
print("\nmodel score: {:.4f} {:.4f}\n".format(score.mean(),score.std()))
"""

In [ ]:
"""
rf_model=RandomForestRegressor(max_depth=5, random_state=42)
rf_model.fit(X,y)
predictions=rf_model.predict(test)
"""

In [ ]:
rf=RandomForestRegressor()
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
output=pd.DataFrame({"id":id,"target":predictions})
output
output.to_csv('sample_submission.csv', index=False)
print("Your submission was successfully saved!")


In [ ]:
output

XGBRegressor
===

In [ ]:
"""import xgboost
from xgboost import XGBRegressor
XGB=XGBRegressor(random_state=42)
XGB.fit(X,y)
predictions=XGB.predict(test)
predictions1=predictions
"""

In [ ]:
"""output=pd.DataFrame({"id":id,"target":predictions})
output
output.to_csv('sample_submission.csv', index=False)
print("Your submission was successfully saved!")
"""

We could try an Averaging model as well

In [ ]:
output

In [ ]:
predictions

In [ ]:
ridge_model=Ridge()
ridge_model.fit(X_train,y_train)
rmsle_cv(ridge_model)

In [ ]:
alphas = [ 1, 5, 10 ,12,13]
cv_ridge = [rmsle_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]

In [ ]:
cv_ridge

In [ ]:
pred_ridge=ridge_model.predict(test)

In [ ]:
pred_ridge

Lightgbm
===

In [ ]:
import lightgbm as lgb
LGB = lgb.LGBMRegressor(random_state=33, n_estimators=5000, min_data_per_group=5, boosting_type='gbdt',
 num_leaves=246, learning_rate=0.005, subsample_for_bin=200000,
 lambda_l1= 1.074622455507616e-05, lambda_l2= 2.0521330798729704e-06, n_jobs=-1, cat_smooth=1.0, 
 importance_type='split', metric='rmse', min_child_samples=20, min_gain_to_split=0.0, feature_fraction=0.5, 
 bagging_freq=6, min_sum_hessian_in_leaf=0.001, min_data_in_leaf=100, bagging_fraction=0.82063411)

LGB.fit(X_train, y_train)


I founded outside the parameters for this model

In [ ]:
predictions=LGB.predict(test)

In [ ]:
submission.id

In [ ]:
output=pd.DataFrame({"id":submission["id"],"target":predictions})
output
output.to_csv('sample_submission.csv', index=False)
print("Your submission was successfully saved!")

**And Finally....**
====

We can see in the graph below the incidence of the different features in the model

This idea was inspired by the Notebook of Imnaho:
https://www.kaggle.com/code/imnaho/eda-predict-with-lgbmclassifier

In [ ]:
plt.figure(figsize=(10,8))

importances=pd.Series(LGB.feature_importances_,index=X.columns)
importances=importances.sort_values()
importances.plot(kind="barh")
plt.title("Feature importances in the LGB model")
plt.show()


In [ ]:
X_train_rs.shape,X_val_rs.shape,y_train.shape,y_val.shape